In [1]:
import os
import json
import boto3
import requests
import pandas as pd
import pyarrow as pa
import awswrangler as wr
import pyarrow.parquet as pq

from io import BytesIO
from datetime import datetime

In [2]:
pd.set_option('display.max_colwidth', None)

In [3]:
# Criar um cliente para o AWS Secrets Manager com base no perfil
client = boto3.session.Session(profile_name='CRISTIAN_AWS').client('secretsmanager', region_name='us-east-1')

# Recuperar o segredo
response = client.get_secret_value(SecretId='env/openweather')
secret_data = response['SecretString']

# Analisar o conteúdo do segredo (assumindo que é um formato JSON)
secrets_dict = json.loads(secret_data)

# Obtenha o valor associado à chave 'OPEN_WEATHER_SECRECT'
open_weather_token = secrets_dict.get('OPEN_WEATHER_SECRECT', '')

In [4]:
API_LINK = f"https://api.openweathermap.org/data/2.5/weather?q=novo hamburgo,br&APPID={open_weather_token}"

In [5]:
requisicao = requests.get(API_LINK)

In [6]:
wheater_json = requisicao.json()

In [7]:
wheater_json

{'coord': {'lon': -51.1306, 'lat': -29.6783},
 'weather': [{'id': 804,
   'main': 'Clouds',
   'description': 'overcast clouds',
   'icon': '04n'}],
 'base': 'stations',
 'main': {'temp': 295.11,
  'feels_like': 295.58,
  'temp_min': 294.86,
  'temp_max': 295.21,
  'pressure': 1014,
  'humidity': 85,
  'sea_level': 1014,
  'grnd_level': 1010},
 'visibility': 10000,
 'wind': {'speed': 2.02, 'deg': 96, 'gust': 5.66},
 'clouds': {'all': 99},
 'dt': 1704418832,
 'sys': {'type': 2,
  'id': 2020383,
  'country': 'BR',
  'sunrise': 1704356966,
  'sunset': 1704407320},
 'timezone': -10800,
 'id': 3456068,
 'name': 'Novo Hamburgo',
 'cod': 200}

In [8]:
# Convertendo o dicionário Python para uma string JSON
weather_json_string = json.dumps(wheater_json)

Save raw JSON at Data-lake

In [9]:
# Defina suas credenciais explicitamente (não recomendado)
profile_name = 'CRISTIAN_AWS'
s3 = boto3.session.Session(profile_name=profile_name).client('s3')
bucket_name = 'data-integration-projects'
directory_path = 'openweather-data-lake/raw-data/'
dia_da_exucucao = datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
file_name = f'weather_data_{dia_da_exucucao}.json'
s3.put_object(Bucket=bucket_name, Key=directory_path + file_name, Body=weather_json_string)

{'ResponseMetadata': {'RequestId': 'YQPB0TDQ8D338ZQZ',
  'HostId': '7Esiu1ogrAG6R4eBwc5AfpM4CNM1y7BE+12tFxXFP8y2PwbpnKg/JKsW5GA9F2n6f+H/HHW89t0=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '7Esiu1ogrAG6R4eBwc5AfpM4CNM1y7BE+12tFxXFP8y2PwbpnKg/JKsW5GA9F2n6f+H/HHW89t0=',
   'x-amz-request-id': 'YQPB0TDQ8D338ZQZ',
   'date': 'Fri, 05 Jan 2024 01:43:32 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"e52535d615365c5d631194111e51c911"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"e52535d615365c5d631194111e51c911"',
 'ServerSideEncryption': 'AES256'}

Import data from data-lake

In [10]:
profile_name = 'CRISTIAN_AWS'

# Configuração do cliente S3
s3 = boto3.session.Session(profile_name=profile_name)
# Substitua 'seu-bucket' e 'seu-arquivo' pelos valores reais
bucket_name = 'data-integration-projects/openweather-data-lake/raw-data'
# Leitura dos dados JSON do S3 usando o awswrangler
df = wr.s3.read_json(f's3://{bucket_name}/*', lines=True, boto3_session = s3)


In [11]:
df.head(5)

,coord,weather,base,main,visibility,wind,clouds,dt,sys,timezone,id,name,cod
0,"{'lon': -51.1306, 'lat': -29.6783}","[{'id': 801, 'main': 'Clouds', 'description': 'few clouds', 'icon': '02n'}]",stations,"{'temp': 293.48, 'feels_like': 293.79, 'temp_min': 293.34, 'temp_max': 294.33, 'pressure': 1013, 'humidity': 85, 'sea_level': 1013, 'grnd_level': 1009}",10000,"{'speed': 1.51, 'deg': 88, 'gust': 4.18}",{'all': 16},1704344172,"{'type': 2, 'id': 2020383, 'country': 'BR', 'sunrise': 1704356966, 'sunset': 1704407320}",-10800,3456068,Novo Hamburgo,200
0,"{'lon': -51.1306, 'lat': -29.6783}","[{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03d'}]",stations,"{'temp': 297.41, 'feels_like': 298.01, 'temp_min': 296.88, 'temp_max': 298.4, 'pressure': 1015, 'humidity': 81, 'sea_level': 1015, 'grnd_level': 1011}",10000,"{'speed': 1.99, 'deg': 108, 'gust': 3.2}",{'all': 35},1704371630,"{'type': 2, 'id': 2020383, 'country': 'BR', 'sunrise': 1704356966, 'sunset': 1704407320}",-10800,3456068,Novo Hamburgo,200
0,"{'lon': -51.1306, 'lat': -29.6783}","[{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03d'}]",stations,"{'temp': 298.46, 'feels_like': 299.06, 'temp_min': 297.94, 'temp_max': 299.68, 'pressure': 1015, 'humidity': 77, 'sea_level': 1015, 'grnd_level': 1011}",10000,"{'speed': 1.99, 'deg': 108, 'gust': 3.2}",{'all': 35},1704373596,"{'type': 2, 'id': 2020383, 'country': 'BR', 'sunrise': 1704356966, 'sunset': 1704407320}",-10800,3456068,Novo Hamburgo,200
0,"{'lon': -51.1306, 'lat': -29.6783}","[{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03d'}]",stations,"{'temp': 299.41, 'feels_like': 299.41, 'temp_min': 297.94, 'temp_max': 300.23, 'pressure': 1015, 'humidity': 71, 'sea_level': 1015, 'grnd_level': 1011}",10000,"{'speed': 2.71, 'deg': 122, 'gust': 3.67}",{'all': 46},1704376514,"{'type': 2, 'id': 2020383, 'country': 'BR', 'sunrise': 1704356966, 'sunset': 1704407320}",-10800,3456068,Novo Hamburgo,200
0,"{'lon': -51.1306, 'lat': -29.6783}","[{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03d'}]",stations,"{'temp': 300.13, 'feels_like': 302.05, 'temp_min': 298.78, 'temp_max': 301.12, 'pressure': 1015, 'humidity': 71, 'sea_level': 1015, 'grnd_level': 1011}",10000,"{'speed': 2.71, 'deg': 122, 'gust': 3.67}",{'all': 46},1704377123,"{'type': 2, 'id': 2020383, 'country': 'BR', 'sunrise': 1704356966, 'sunset': 1704407320}",-10800,3456068,Novo Hamburgo,200


In [12]:
df['lon'] = df['coord'].apply(lambda x: x['lon'])
df['lat'] = df['coord'].apply(lambda x: x['lat'])

In [13]:
df['clouds_description'] = df['weather'].apply(lambda x: x[0]['description'] if x and isinstance(x, list) and 'description' in x[0] else None)

In [14]:
df['temp'] = df['main'].apply(lambda x: x['temp'])
df['feels_like'] = df['main'].apply(lambda x: x['feels_like'])
df['temp_min'] = df['main'].apply(lambda x: x['temp_min'])
df['temp_max'] = df['main'].apply(lambda x: x['temp_max'])
df['pressure'] = df['main'].apply(lambda x: x['pressure'])
df['humidity'] = df['main'].apply(lambda x: x['humidity'])

In [15]:
df['clouds_number'] = df['clouds'].apply(lambda x: x['all'])

In [16]:
df['country'] = df['sys'].apply(lambda x: x['country'])

In [17]:
df['dt'] = pd.to_numeric(df['dt'])

# Converta para formato de data e hora
df['dt_utc'] = pd.to_datetime(df['dt'], unit='s')

# Converta para UTC
df['dt_utc'] = pd.to_datetime(df['dt_utc'], utc=True)

In [18]:
df.drop(columns= ["coord", "weather", "main", "wind", "clouds", "sys", "dt", "timezone"], inplace= True)

In [19]:
df.reset_index(drop= True)

,base,visibility,id,name,cod,lon,lat,clouds_description,temp,feels_like,temp_min,temp_max,pressure,humidity,clouds_number,country,dt_utc
0,stations,10000,3456068,Novo Hamburgo,200,-51.1306,-29.6783,few clouds,293.48,293.79,293.34,294.33,1013,85,16,BR,2024-01-04 04:56:12+00:00
1,stations,10000,3456068,Novo Hamburgo,200,-51.1306,-29.6783,scattered clouds,297.41,298.01,296.88,298.40,1015,81,35,BR,2024-01-04 12:33:50+00:00
2,stations,10000,3456068,Novo Hamburgo,200,-51.1306,-29.6783,scattered clouds,298.46,299.06,297.94,299.68,1015,77,35,BR,2024-01-04 13:06:36+00:00
3,stations,10000,3456068,Novo Hamburgo,200,-51.1306,-29.6783,scattered clouds,299.41,299.41,297.94,300.23,1015,71,46,BR,2024-01-04 13:55:14+00:00
4,stations,10000,3456068,Novo Hamburgo,200,-51.1306,-29.6783,scattered clouds,300.13,302.05,298.78,301.12,1015,71,46,BR,2024-01-04 14:05:23+00:00
5,stations,10000,3456068,Novo Hamburgo,200,-51.1306,-29.6783,scattered clouds,300.13,302.05,298.78,301.12,1015,71,46,BR,2024-01-04 14:05:23+00:00
6,stations,10000,3456068,Novo Hamburgo,200,-51.1306,-29.6783,scattered clouds,299.78,299.78,297.97,301.12,1015,71,46,BR,2024-01-04 14:15:14+00:00
7,stations,10000,3456068,Novo Hamburgo,200,-51.1306,-29.6783,scattered clouds,299.78,299.78,297.97,301.12,1015,71,46,BR,2024-01-04 14:15:14+00:00
8,stations,10000,3456068,Novo Hamburgo,200,-51.1306,-29.6783,broken clouds,300.07,301.95,297.97,301.90,1014,71,51,BR,2024-01-04 14:54:09+00:00
9,stations,10000,3456068,Novo Hamburgo,200,-51.1306,-29.6783,broken clouds,300.07,301.95,297.97,301.90,1014,71,51,BR,2024-01-04 14:54:09+00:00


In [20]:
df.head(5)

,base,visibility,id,name,cod,lon,lat,clouds_description,temp,feels_like,temp_min,temp_max,pressure,humidity,clouds_number,country,dt_utc
0,stations,10000,3456068,Novo Hamburgo,200,-51.1306,-29.6783,few clouds,293.48,293.79,293.34,294.33,1013,85,16,BR,2024-01-04 04:56:12+00:00
0,stations,10000,3456068,Novo Hamburgo,200,-51.1306,-29.6783,scattered clouds,297.41,298.01,296.88,298.40,1015,81,35,BR,2024-01-04 12:33:50+00:00
0,stations,10000,3456068,Novo Hamburgo,200,-51.1306,-29.6783,scattered clouds,298.46,299.06,297.94,299.68,1015,77,35,BR,2024-01-04 13:06:36+00:00
0,stations,10000,3456068,Novo Hamburgo,200,-51.1306,-29.6783,scattered clouds,299.41,299.41,297.94,300.23,1015,71,46,BR,2024-01-04 13:55:14+00:00
0,stations,10000,3456068,Novo Hamburgo,200,-51.1306,-29.6783,scattered clouds,300.13,302.05,298.78,301.12,1015,71,46,BR,2024-01-04 14:05:23+00:00


Save at S3

In [21]:
# Defina suas credenciais explicitamente (não recomendado)
profile_name = 'CRISTIAN_AWS'
s3 = boto3.session.Session(profile_name=profile_name).client('s3')
bucket_name = 'data-integration-projects'
directory_path = 'openweather-data-lake/processed-data/'
dia_da_exucucao = datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
file_name = f'df{dia_da_exucucao}.parquet'
df.to_parquet(file_name)
s3.upload_file(file_name, bucket_name, directory_path + file_name)